In [67]:
import pandas as pd

In [68]:
csv_1 = pd.read_csv('neutral.csv', index_col=0, sep=',', encoding='utf-8')

In [69]:

import ast
csv_1['hypothesis'] = csv_1['hypothesis'].apply(ast.literal_eval)
csv_1

,Unnamed: 0,premise,hypothesis,prompt,rationale
0,0.0,If you catch your girlfriend lying about seein...,"[broken heart, mistrust, getting dumped, being...",Questions: If you catch your girlfriend lying ...,1. The correct answer is **(B) mistrust**. If ...
1,1.0,"If you choose to fly in an airplane, it normal...","[like bird, get home, go half an hour away, go...",Questions: If you choose to fly in an airplane...,2. The correct answer is **(E) go far**. If yo...
2,2.0,"If you cut when waiting at the ticket office, ...","[auditorium, humans, opera house, death, end o...",Questions: If you cut when waiting at the tick...,3. The correct answer is **(E) end of line**. ...
3,3.0,"If you did well on the test, you might get an ...","[see work, practice, quiz, school grade, homew...","Questions: If you did well on the test, you mi...",4. The correct answer is **(D) school grade**....
4,4.0,If you didn't find any plants on another plane...,"[surface of earth, park, greenhouse, windowsil...",Questions: If you didn't find any plants on an...,5. The correct answer is **(A) surface of eart...
...,...,...,...,...,...
9736,NaN,Learning languages is difficult. Even in a fu...,"[better communication, overthinking, frustrati...",Questions: Learning languages is difficult. E...,4. The correct answer is **(C) frustration**. ...
9737,NaN,Lilly was still getting warm. She ran her han...,"[stroke, pleasure, increase of temperature, ge...",Questions: Lilly was still getting warm. She ...,5. The correct answer is **(B) pleasure**. Lil...
9738,NaN,"Lilly went bathing fife times a day, but this ...","[cleanliness, dry skin, skin damage, nudity, t...",Questions: Lilly went bathing fife times a day...,6. The correct answer is **(B) dry skin**. Lil...
9739,NaN,The couple was attempting procreating night af...,"[economic growth, population increase, exhaust...",Questions: The couple was attempting procreati...,7. The correct answer is **(C) exhaustion**. T...


In [662]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [70]:
prompt_template= '''
Questions: %s \n
Choices: (A) %s, (B) %s, (C) %s, (D) %s, (E) %s \
What is the commonly agreed-upon answer to the question '%s' with options %s, %s, %s, %s %s? Justify your answer based on general knowledge. \n
'''

In [71]:
error = csv_1[csv_1.rationale.isna()]

In [665]:
# error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'][0], x['hypothesis'][1], x['hypothesis'][2], x['hypothesis'][3], x['hypothesis'][4],
#                                                           x['premise'], x['hypothesis'][0], x['hypothesis'][1], x['hypothesis'][2], x['hypothesis'][3], x['hypothesis'][4]), axis=1)


In [72]:
csv_1.rationale = csv_1.rationale.astype(str)

In [73]:
# check if length of hypothesis is less than 5
error['len'] = csv_1['rationale'].apply(lambda x: len(x))
error = error[error['len'] < 5]

<ipython-input-73-b24a8e28a7c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['len'] = csv_1['rationale'].apply(lambda x: len(x))


In [74]:
csv_1.loc[error.index].reset_index(drop=True).to_csv('error.csv')

In [76]:
error.index

Int64Index([9600], dtype='int64')

In [77]:
print(error.values)

[[nan nan nan nan nan 1]]


In [78]:
csv_1['split'] = csv_1.rationale.apply(lambda x: x.lower().split('most likely answer'))

In [83]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [81]:
after_handle = pd.read_csv('error.csv', index_col=0, sep='|', encoding='utf-8')

# replace the csv_1 with after_handle
csv_1.loc[after_handle.index, 'rationale'] = after_handle['rationale']

KeyError: 'rationale'

In [94]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in row['hypothesis']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [87]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in row['hypothesis']:
        if choice in row['rationale'].lower():
            row.loc['correct_index'][choice] = row['rationale'].lower().index(choice)

In [95]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [96]:
csv_1[csv_1.LLM_answer.isna()]

,Unnamed: 0,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer
0,0.0,If you catch your girlfriend lying about seein...,"[broken heart, mistrust, getting dumped, being...",Questions: If you catch your girlfriend lying ...,1. The correct answer is **(B) mistrust**. If ...,.,{},NaN
1,1.0,"If you choose to fly in an airplane, it normal...","[like bird, get home, go half an hour away, go...",Questions: If you choose to fly in an airplane...,2. The correct answer is **(E) go far**. If yo...,.,{},NaN
2,2.0,"If you cut when waiting at the ticket office, ...","[auditorium, humans, opera house, death, end o...",Questions: If you cut when waiting at the tick...,3. The correct answer is **(E) end of line**. ...,.,{},NaN
3,3.0,"If you did well on the test, you might get an ...","[see work, practice, quiz, school grade, homew...","Questions: If you did well on the test, you mi...",4. The correct answer is **(D) school grade**....,.,{},NaN
4,4.0,If you didn't find any plants on another plane...,"[surface of earth, park, greenhouse, windowsil...",Questions: If you didn't find any plants on an...,5. The correct answer is **(A) surface of eart...,.,{},NaN
...,...,...,...,...,...,...,...,...
9736,NaN,Learning languages is difficult. Even in a fu...,"[better communication, overthinking, frustrati...",Questions: Learning languages is difficult. E...,4. The correct answer is **(C) frustration**. ...,.,{},NaN
9737,NaN,Lilly was still getting warm. She ran her han...,"[stroke, pleasure, increase of temperature, ge...",Questions: Lilly was still getting warm. She ...,5. The correct answer is **(B) pleasure**. Lil...,.,{},NaN
9738,NaN,"Lilly went bathing fife times a day, but this ...","[cleanliness, dry skin, skin damage, nudity, t...",Questions: Lilly went bathing fife times a day...,6. The correct answer is **(B) dry skin**. Lil...,.,{},NaN
9739,NaN,The couple was attempting procreating night af...,"[economic growth, population increase, exhaust...",Questions: The couple was attempting procreati...,7. The correct answer is **(C) exhaustion**. T...,.,{},NaN


In [744]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [680]:
paper = pd.read_csv('paper.csv', index_col=0)

In [689]:
csv_1['question'] = csv_1.apply(lambda x: f'{x.premise}\nAnswer Choices:\n(a) {x.hypothesis[0]}\n(b) {x.hypothesis[1]}\n(x.hypothesis[ ]{x.hypothesis[2]}\n(d) {x.hypothesis[3]}\n(e) {x.hypothesis[4]}', axis=1)
csv_1.set_index('question', inplace=True)

In [715]:
paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index('premise', inplace=True)

In [718]:
total_df = csv_1[(csv_1.premise.isin(paper.premise))]
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"""There are 10 apples on an apple tree. Three fall off. Now there are X apples."" What is this an example of?\nAnswer Choices:\n(a) park\n(b) coloring book\n(x.hypothesis[ ]garden center\n(d) math problem\n(e) gravity","""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...",1. The commonly agreed-upon answer to the ques...,1. the commonly agreed-upon answer to the ques...,{'math problem': 163},math problem
"A John is a bum. Much like the stereotype, he lives near this sort of transportation infrastructure. Where does he live?\nAnswer Choices:\n(a) bus depot\n(b) beach\n(x.hypothesis[ ]train station\n(d) bridge\n(e) bridge","A John is a bum. Much like the stereotype, he...","[bus depot, beach, train station, bridge, bridge]",2. The commonly agreed-upon answer to the ques...,2. the commonly agreed-upon answer to the ques...,{'bridge': 179},bridge
"A bad person places little value on being honest, acting without pretense or being what?\nAnswer Choices:\n(a) excellent\n(b) upright\n(x.hypothesis[ ]premium\n(d) competent\n(e) sincere",A bad person places little value on being hone...,"[excellent, upright, premium, competent, sincere]",3. The commonly agreed-upon answer to the ques...,3. the commonly agreed-upon answer to the ques...,{'sincere': 145},sincere
"A bald eagle flies over St. Paul, where is it?\nAnswer Choices:\n(a) texas\n(b) thermal\n(x.hypothesis[ ]minnesota\n(d) canada\n(e) photograph","A bald eagle flies over St. Paul, where is it?","[texas, thermal, minnesota, canada, photograph]",4. The commonly agreed-upon answer to the ques...,4. the commonly agreed-upon answer to the ques...,{'minnesota': 103},minnesota
"A battleship is a powerful vessel. If you need something similar but faster, what would you use?\nAnswer Choices:\n(a) yatch\n(b) corvette\n(x.hypothesis[ ]aircraft carrier\n(d) destroyer\n(e) patrol boat",A battleship is a powerful vessel. If you nee...,"[yatch, corvette, aircraft carrier, destroyer,...",5. The commonly agreed-upon answer to the ques...,5. the commonly agreed-upon answer to the ques...,{'destroyer': 155},destroyer
...,...,...,...,...,...,...
where is the information superhighway?\nAnswer Choices:\n(a) heavily travelled area\n(b) cyberspace\n(x.hypothesis[ ]industrialized country\n(d) on the internet.\n(e) computer network,where is the information superhighway?,"[heavily travelled area, cyberspace, industria...",7. The commonly agreed-upon answer to the ques...,7. the commonly agreed-upon answer to the ques...,{'cyberspace': 99},cyberspace
where would a good newspaper boy put a paper in the rain?\nAnswer Choices:\n(a) library\n(b) lawn\n(x.hypothesis[ ]trash\n(d) roof\n(e) front door,where would a good newspaper boy put a paper i...,"[library, lawn, trash, roof, front door]",8. The commonly agreed-upon answer to the ques...,8. the commonly agreed-upon answer to the ques...,{'front door': 118},front door
which door with lock regularly holds cars?\nAnswer Choices:\n(a) house\n(b) autolock\n(x.hypothesis[ ]garage\n(d) file cabinet\n(e) safe,which door with lock regularly holds cars?,"[house, autolock, garage, file cabinet, safe]",9. The commonly agreed-upon answer to the ques...,9. the commonly agreed-upon answer to the ques...,{'garage': 103},garage


In [724]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"""There are 10 apples on an apple tree. Three fall off. Now there are X apples."" What is this an example of?\nAnswer Choices:\n(a) park\n(b) coloring book\n(x.hypothesis[ ]garden center\n(d) math problem\n(e) gravity","""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...",1. The commonly agreed-upon answer to the ques...,1. the commonly agreed-upon answer to the ques...,{'math problem': 163},math problem
"A John is a bum. Much like the stereotype, he lives near this sort of transportation infrastructure. Where does he live?\nAnswer Choices:\n(a) bus depot\n(b) beach\n(x.hypothesis[ ]train station\n(d) bridge\n(e) bridge","A John is a bum. Much like the stereotype, he...","[bus depot, beach, train station, bridge, bridge]",2. The commonly agreed-upon answer to the ques...,2. the commonly agreed-upon answer to the ques...,{'bridge': 179},bridge
"A bad person places little value on being honest, acting without pretense or being what?\nAnswer Choices:\n(a) excellent\n(b) upright\n(x.hypothesis[ ]premium\n(d) competent\n(e) sincere",A bad person places little value on being hone...,"[excellent, upright, premium, competent, sincere]",3. The commonly agreed-upon answer to the ques...,3. the commonly agreed-upon answer to the ques...,{'sincere': 145},sincere
"A bald eagle flies over St. Paul, where is it?\nAnswer Choices:\n(a) texas\n(b) thermal\n(x.hypothesis[ ]minnesota\n(d) canada\n(e) photograph","A bald eagle flies over St. Paul, where is it?","[texas, thermal, minnesota, canada, photograph]",4. The commonly agreed-upon answer to the ques...,4. the commonly agreed-upon answer to the ques...,{'minnesota': 103},minnesota
"A battleship is a powerful vessel. If you need something similar but faster, what would you use?\nAnswer Choices:\n(a) yatch\n(b) corvette\n(x.hypothesis[ ]aircraft carrier\n(d) destroyer\n(e) patrol boat",A battleship is a powerful vessel. If you nee...,"[yatch, corvette, aircraft carrier, destroyer,...",5. The commonly agreed-upon answer to the ques...,5. the commonly agreed-upon answer to the ques...,{'destroyer': 155},destroyer
...,...,...,...,...,...,...
where is the information superhighway?\nAnswer Choices:\n(a) heavily travelled area\n(b) cyberspace\n(x.hypothesis[ ]industrialized country\n(d) on the internet.\n(e) computer network,where is the information superhighway?,"[heavily travelled area, cyberspace, industria...",7. The commonly agreed-upon answer to the ques...,7. the commonly agreed-upon answer to the ques...,{'cyberspace': 99},cyberspace
where would a good newspaper boy put a paper in the rain?\nAnswer Choices:\n(a) library\n(b) lawn\n(x.hypothesis[ ]trash\n(d) roof\n(e) front door,where would a good newspaper boy put a paper i...,"[library, lawn, trash, roof, front door]",8. The commonly agreed-upon answer to the ques...,8. the commonly agreed-upon answer to the ques...,{'front door': 118},front door
which door with lock regularly holds cars?\nAnswer Choices:\n(a) house\n(b) autolock\n(x.hypothesis[ ]garage\n(d) file cabinet\n(e) safe,which door with lock regularly holds cars?,"[house, autolock, garage, file cabinet, safe]",9. The commonly agreed-upon answer to the ques...,9. the commonly agreed-upon answer to the ques...,{'garage': 103},garage


In [727]:
paper.set_index('premise', inplace=True)
total_df.set_index('premise', inplace=True)
paper['my_label'] = total_df['LLM_answer']

In [728]:
paper

,input,label,llm_label,rationale,my_label
premise,,,,,
"A kid is studying new words at home in the kitchen, where could a dictionary be?",A kid is studying new words at home in the kit...,table,library,The answer must be a place where a dictionary ...,shelf
Sarah did all of it. She was the best saleswoman in the city. But she was never acknowledge for her what?,Sarah did all of it. She was the best saleswo...,work hard,productivity,The answer must be something that Sarah did th...,productivity
Where do you bring your flight bag?,Where do you bring your flight bag?\nAnswer Ch...,airport,airport,The answer must be the place where you bring y...,airport
when im competing in a competition what should i aim for,when im competing in a competition what should...,wins,wins,The answer should be the goal of someone compe...,success
"The computer stopped working, even though John tried his best. What happened to it?","The computer stopped working, even though John...",crash,crash,The answer must be something that happened to ...,crash
...,...,...,...,...,...
"If people need something on the other side of a body of water, what must they do?",If people need something on the other side of ...,cross river,cross river,The answer must be something that people do to...,cross river
What kind of pain can be caused my watching television too much?,What kind of pain can be caused my watching te...,headaches,headaches,The answer must be a pain caused by watching t...,headaches
"The tourists swam in a swimming pool, isolated from the rest of the country, where were they?","The tourists swam in a swimming pool, isolated...",resort hotel,resort hotel,The answer must be a place where tourists can ...,resort hotel


In [729]:
sum(paper.label == paper.my_label) / len(paper)

0.7465352633199877

In [730]:
sum(paper.label == paper.llm_label) / len(paper)

0.7426342264654553

In [41]:
total_df.to_csv('consensus - full.csv', encoding='utf-8')